# Bedrock AgentCore Code Interpreter를 사용한 Strands Agents

이 워크샵은 Strands Agents를 Amazon Bedrock AgentCore Code Interpreter와 통합하여 코드를 동적으로 실행할 수 있는 AI 에이전트를 생성하는 방법을 보여줍니다.

## 개요

이 실습에서는 다음을 수행합니다:
- Bedrock AgentCore Code Interpreter 기능에 대해 학습
- Strands Agents와 기본 Code Interpreter 테스트
- 공용 네트워크 액세스가 가능한 커스텀 Code Interpreter 생성
- 다양한 실행 환경과 그 제한사항 비교

## 사전 요구사항

이 실습을 시작하기 전에 다음 사항을 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Strands Agents와 Bedrock AgentCore Python SDK에 필요한 패키지를 설치하세요:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore rich

AWS 리전에 따라 Nova Pro 모델 ID를 설정하세요:

In [2]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

Nova Pro Model ID: us.amazon.nova-pro-v1:0


## Bedrock AgentCore Code Interpreter란 무엇인가요?

Amazon Bedrock AgentCore Code Interpreter는 AI 에이전트가 보안 샌드박스 환경에서 코드를 동적으로 실행할 수 있게 해주는 강력한 도구입니다. 주요 이점은 다음과 같습니다:

- **보안 실행**: 격리된 샌드박스 환경에서 Python 코드 실행
- **동적 문제 해결**: 에이전트가 계산, 데이터 분석, 시각화 생성을 수행할 수 있게 함
- **유연한 구성**: 기본 샌드박스와 커스텀 네트워크 지원 환경 모두 지원
- **통합 준비**: Strands Agents 및 기타 AI 프레임워크와 원활하게 통합

Code Interpreter는 에이전트에게 계산 분석, 데이터 처리 또는 수학적 계산이 필요한 복잡한 문제를 해결할 수 있는 능력을 제공합니다.

### 기본 Code Interpreter를 사용한 Strands Agent 테스트

기본 AgentCore Code Interpreter를 사용하여 Strands Agent를 테스트하는 것부터 시작해보겠습니다. 

샌드박스 환경에서 수학 문제를 해결하기 위해 코드를 생성하고 실행하는 방법을 보여드리겠습니다.

In [3]:
from strands import Agent
from strands.models import BedrockModel
from strands_tools.code_interpreter import AgentCoreCodeInterpreter

agentcore_code_interpreter = AgentCoreCodeInterpreter()

# 코드 생성 어시스턴트 에이전트 생성
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="""당신은 코드 실행을 통해 모든 답변을 검증하는 도움이 되는 AI 어시스턴트입니다.""",
    tools=[agentcore_code_interpreter.code_interpreter],
)

agent("반지름이 8.26cm인 원의 넓이는 얼마인가요?")

<thinking> To calculate the area of a circle, we use the formula: Area = π * radius^2. Given the radius is 8.26 cm, I will use the code interpreter tool to perform the calculation. </thinking>

Tool #1: code_interpreter
<thinking> It seems that the session was not found, so I will create a new session and execute the code to calculate the area of the circle. </thinking> 
Tool #2: code_interpreter

Tool #3: code_interpreter
<thinking> It seems that there was an issue with the session, so I will try executing the code again to calculate the area of the circle. </thinking> 
Tool #4: code_interpreter
<thinking> It seems there was a syntax error in the code due to incorrect line continuation. I will correct the code and try again. </thinking> 
Tool #5: code_interpreter
<thinking> The syntax error persists, indicating an issue with the line continuation character. I will correct the code by removing the backslashes and try again. </thinking> 
Tool #6: code_interpreter
<thinking> The calculat

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '<thinking> The calculation was successful, and the area of the circle with a radius of 8.26 cm is approximately 214.34 square centimeters. I will now inform the user of the result. </thinking>\n\n반지름이 8.26cm인 원의 넓이는 약 214.34제곱센티미터입니다.'}]}, metrics=EventLoopMetrics(cycle_count=6, tool_metrics={'code_interpreter': ToolMetrics(tool={'toolUseId': 'tooluse_IQbx3fB3RKyxWt4SPpF-AA', 'name': 'code_interpreter', 'input': {'code_interpreter_input': {'action': {'session_name': 'circle_area_session', 'language': 'python', 'code': 'import math\nradius = 8.26\narea = math.pi * radius**2\nprint(area)', 'type': 'executeCode'}}}}, call_count=6, success_count=4, error_count=2, total_time=1.4289329051971436)}, cycle_durations=[2.04738712310791], traces=[<strands.telemetry.metrics.Trace object at 0xffff6f3d0090>, <strands.telemetry.metrics.Trace object at 0xffff88bd6210>, <strands.telemetry.metrics.Trace object at 0xff

에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:

agent 가 자동으로 code interpreter 환경을 만들고 code 를 생성하고 실행하는 것을 볼 수 있습니다. 

In [4]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 횟수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

에이전트 루프 세부사항

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

루프 횟수: 6

                                                  에이전트 메시지                                                  
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 역할      ┃ 텍스트                    ┃ 도구 이름        ┃ 도구 입력                 ┃ 도구 결과                ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user      │ 반지름이 8.26cm인 원의    │                  │                           │                          │
│           │ 넓이는 얼마인가요?        │                  │                           │                          │
├───────────┼───────────────────────────┼──────────────────┼───────────────────────────┼──────────────────────────┤
│ assistant │ <thinking> To calculate   │ code_interpreter │ {                         │                          │
│           │ the area of a circle, we  │                  │   "code_interpreter_inpu… │                          │
│           │ use the formula: Area = π │                  │ {                         │                          │
│           │ * radius^2. Given the     │                  │     "action": {           │                          │
│           │ radius is 8.26 cm, I will │                  │       "type":             │                          │
│           │ use the code interpreter  │                  │ "executeCode",            │                          │
│           │ tool to perform the       │                  │       "session_name":     │                          │
│           │ calculation. </thinking>  │                  │ "circle_area_session",    │                          │
│           │                           │                  │       "code": "import     │                          │
│           │                           │                  │ math\\nradius =           │                          │
│           │                           │                  │ 8.26\\narea = math.pi *   │                          │
│           │                           │                  │ radius**2\\nprint(area)", │                          │
│           │                           │                  │       "language":         │                          │
│           │                           │                  │ "python"                  │                          │
│           │                           │                  │     }                     │                          │
│           │                           │                  │   }                       │                          │
│           │                           │                  │ }                         │                          │
├───────────┼───────────────────────────┼──────────────────┼───────────────────────────┼──────────────────────────┤
│ user      │                           │                  │                           │ {                        │
│           │                           │                  │                           │   "text": "Session       │
│           │                           │                  │                           │ 'circle_area_session'    │
│           │                           │                  │                           │ not found"               │
│           │                           │                  │                           │ }                        │
├───────────┼───────────────────────────┼──────────────────┼───────────────────────────┼──────────────────────────┤
│ assistant │ <thinking> It seems that  │ code_interpreter │ {                         │                          │
│           │ the session was not       │                  │   "code_interpreter_inpu… │                          │
│           │ found, so I will create a │                  │ {                         │                          │
│           │ new session and execute   │                  │     "action": {           │       

### 샌드박스 환경 제한사항 이해

기본 Code Interpreter는 **네트워크 격리가 있는 샌드박스 환경**에서 작동합니다. 

이는 코드 실행 안전성을 보장하면서 무단 네트워크 액세스를 방지하는 중요한 보안 기능입니다.

## 네트워크 액세스가 가능한 커스텀 Code Interpreter 생성

웹 기반 작업을 활성화하기 위해 공용 네트워크 액세스가 가능한 커스텀 Code Interpreter를 생성하겠습니다. 

이는 다양한 사용 사례에 대한 AgentCore 플랫폼의 유연성을 보여줍니다.

### 1단계: AgentCore 클라이언트 초기화

먼저 control plane 과 data plane 작업 모두에 필요한 클라이언트를 설정해보겠습니다:

Control Plane Endpoint 는 Gateway, Memory, Code Interpreter 등의 리소스를 생성, 수정, 삭제하는 CRUDL(Create, Read, Update, Delete, List) 작업을 처리합니다

Data Plane Endpoint 는 Agent 호출, Tool 실행, 코드 실행 등 실제 운영 작업을 처리합니다

In [5]:
from bedrock_agentcore._utils import endpoints
import boto3
import json

region = boto3.session.Session().region_name

data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

### 2단계: 커스텀 Code Interpreter 생성

공용 네트워크 액세스가 가능한 커스텀 Code Interpreter를 생성합니다:

이 때는 control plane endpoint 에 대해 public network 접근이 가능하도록 지정한 후 code interpreter 를 생성합니다.

In [6]:
from botocore.exceptions import ClientError

interpreter_name = "SampleCodeInterpreter"

# 코드 인터프리터 생성
try:
    interpreter_response = cp_client.create_code_interpreter(
        name=interpreter_name,
        description="Code Interpreter 샘플 테스트를 위한 환경",
        #executionRoleArn=iam_role_arn, #코드 인터프리터가 AWS 리소스에 액세스해야 하는 경우에만 필요
        networkConfiguration={
            'networkMode': 'PUBLIC'
        }
    )
    interpreter_id = interpreter_response["codeInterpreterId"]
    print(f"인터프리터 생성됨: {interpreter_id}")
except ClientError as e:
    print(f"오류: {e}")
    if "already exists" in str(e):
        # 코드 인터프리터가 이미 존재하는 경우 ID 검색
        for items in cp_client.list_code_interpreters()['codeInterpreterSummaries']:
            if items['name'] == interpreter_name:
                interpreter_id = items['codeInterpreterId']
                print(f"Code Interpreter ID: {interpreter_id}")
                break
except Exception as e:
    # 코드 인터프리터 생성 중 오류 표시
    print(f"오류: {e}")

인터프리터 생성됨: SampleCodeInterpreter-gGSqhHgruR


### 3단계: Code Interpreter 세션 생성

커스텀 Code Interpreter에서 코드 인터프리터 세션을 생성합니다:

data plane endpoint 를 호출하여 control plane 에서 생성된 인터프리터를 사용하도록 세션을 생성합니다.

In [7]:
from botocore.exceptions import ClientError

session_name = "SampleCodeInterpreterSession"

# 코드 인터프리터 세션 생성
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name=session_name,
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"세션 생성됨: {session_id}")

세션 생성됨: 01KEM6QRQ4RJJ72WZ9KWTRPD80


### 4단계: 기본 기능 테스트

코드 인터프리터 세션이 pip로 Yahoo Finance Python 패키지를 설치하고 해당 패키지를 사용하여 오늘 Amazon 주가를 검색함으로써 인터넷에 액세스할 수 있는지 확인해보겠습니다:

In [8]:
# code interpreter 환경에 python package 를 설치합니다.
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "pip install yfinance"
    }
)

# code interpreter 환경에 python code 를 전달하고 code 를 실행하도록 합니다.
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCode",
    arguments={"code": """
        import yfinance as yf
               
        amzn = yf.Ticker('AMZN')
        data = amzn.history(period='1d')
        today_close = data['Close'][-1]
        print(today_close)
        """,
        "language": "python",
        "clearContext": False
    }
)

for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

{
  "content": [
    {
      "type": "text",
      "text": "247.3800048828125"
    },
    {
      "type": "text",
      "text": "/tmp/ipykernel_299/3027036176.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`\n  today_close = data['Close'][-1]"
    }
  ],
  "structuredContent": {
    "stdout": "247.3800048828125",
    "stderr": "/tmp/ipykernel_299/3027036176.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`\n  today_close = data['Close'][-1]",
    "exitCode": 0,
    "executionTime": 1.3446993827819824
  },
  "isError": false
}


## Strands Agent와 커스텀 Code Interpreter 사용

위에서는 직접 코드를 전달했지만 Code Interpreter 에게 code 를 생성하도록 하고 이것을 도구로 등록하여 사용할 수 있습니다.

이제 인터넷 액세스가 가능한 커스텀 Code Interpreter를 Strands Agent에 통합하겠습니다. 

향상된 기능을 위해 동일한 코드 인터프리터 세션을 공유하는 커스텀 `execute_python` 및 `execute_command` 도구를 생성하겠습니다.

아래 system_prompt 는 한글로 전달할 경우 잘 처리하지 못합니다.  Nova Pro 의 경우 사용자 질문은 한글로 하더라도 system prompt 는 영문으로 하는 것이 좋습니다. 다만 Bedrock 에서 제공하는 Claude 모델은 한글 system prompt 도 잘 처리합니다. 

In [11]:
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import boto3

# 위에서 생성한 Core Interpreter와 Session 재사용
ci_client = CodeInterpreter(region=boto3.session.Session().region_name)
ci_client.start(identifier=interpreter_id) # 새로운 코드 인터프리터 세션 초기화

@tool
def execute_python(code: str, description: str = "") -> str:
    """샌드박스에서 Python 코드를 실행합니다."""
    
    print(f"\n 생성된 코드: {code}")
    response = ci_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
        
    for event in response["stream"]:
        return json.dumps(event["result"])

@tool
def execute_command(command: str, description: str = "") -> str:
    """샌드박스에서 명령을 실행합니다."""
    
    print(f"\n 생성된 명령: {command}")
    response = ci_client.invoke("executeCommand", {
        "command": command
    })
    
    for event in response["stream"]:
        return json.dumps(event["result"])
    

# 코드 생성 에이전트 생성
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.
                  If you has no available tools to perform the task, you must generate and execute code to continue. 
                  If required, execute pip install to download required packages.
                  """,
    tools=[execute_python, execute_command],
)

agent("오늘 Amazon의 주가는 얼마인가요?")

ci_client.stop() # 현재 코드 인터프리터 세션 중지

<thinking> To obtain the current stock price of Amazon, I need to use a financial data library such as `yfinance`. I will first check if the `yfinance` library is installed. If not, I will install it. Then, I will use the library to fetch the current stock price of Amazon. </thinking>

Tool #1: execute_command

 생성된 명령: pip install yfinance
<thinking> The `yfinance` library has been successfully installed. Now I will use this library to fetch the current stock price of Amazon. </thinking> 
Tool #2: execute_python

 생성된 코드: import yfinance as yf

# Fetch current stock price of Amazon
amzn = yf.Ticker('AMZN')
current_price = amzn.info['regularMarketPrice']
current_price
현재 Amazon의 주가는 247.38 USD입니다.

에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:

In [12]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 횟수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

에이전트 루프 세부사항

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

루프 횟수: 3

                                                  에이전트 메시지                                                  
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 역할      ┃ 텍스트                    ┃ 도구 이름       ┃ 도구 입력                 ┃ 도구 결과                 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user      │ 오늘 Amazon의 주가는      │                 │                           │                           │
│           │ 얼마인가요?               │                 │                           │                           │
├───────────┼───────────────────────────┼─────────────────┼───────────────────────────┼───────────────────────────┤
│ assistant │ <thinking> To obtain the  │ execute_command │ {                         │                           │
│           │ current stock price of    │                 │   "command": "pip install │                           │
│           │ Amazon, I need to use a   │                 │ yfinance"                 │                           │
│           │ financial data library    │                 │ }                         │                           │
│           │ such as `yfinance`. I     │                 │                           │                           │
│           │ will first check if the   │                 │                           │                           │
│           │ `yfinance` library is     │                 │                           │                           │
│           │ installed. If not, I will │                 │                           │                           │
│           │ install it. Then, I will  │                 │                           │                           │
│           │ use the library to fetch  │                 │                           │                           │
│           │ the current stock price   │                 │                           │                           │
│           │ of Amazon. </thinking>    │                 │                           │                           │
│           │                           │                 │                           │                           │
├───────────┼───────────────────────────┼─────────────────┼───────────────────────────┼───────────────────────────┤
│ user      │                           │                 │                           │ {                         │
│           │                           │                 │                           │   "text": "{\"content\":  │
│           │                           │                 │                           │ [{\"type\": \"text\",     │
│           │                           │                 │                           │ \"text\": \"Collecting    │
│           │                           │                 │                           │ yfinance\\r\\n            │
│           │                           │                 │                           │ Downloading               │
│           │                           │                 │                           │ yfinance-1.0-py2.py3-non… │
│           │                           │                 │                           │ (6.0 kB)\\r\\nRequirement │
│           │                           │                 │                           │ already satisfied:        │
│           │                           │                 │                           │ pandas>=1.3.0 in          │
│           │                           │                 │                           │ /opt/amazon/genesis1p-to… │
│           │                           │                 │                           │ (from yfinance)           │
│           │                           │                 │                           │ (2.3.1)\\r\\nRequirement  │
│           │                           │                 │                           │ alr

## 리소스 정리 (선택사항)

불필요한 요금을 피하기 위해 생성한 AgentCore Runtime 리소스를 정리합니다:

In [ ]:
import boto3

cp_client = boto3.client("bedrock-agentcore-control", region_name=region, endpoint_url=control_plane_endpoint)
dp_client = boto3.client("bedrock-agentcore", region_name=region, endpoint_url=data_plane_endpoint)

try:
    print("세션과 인터프리터를 정리하는 중...")
    dp_client.stop_code_interpreter_session(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id
    )
    print("✓ 세션이 성공적으로 중지되었습니다")

    cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
    print("✓ 인터프리터가 성공적으로 삭제되었습니다")
except Exception as e:
    print(f"❌ 정리 중 오류 발생: {e}")
    print("일부 리소스를 수동으로 정리해야 할 수 있습니다.")

## 결론

이 실습에서 성공적으로 다음을 수행했습니다:

- ✅ 기본 Bedrock AgentCore Code Interpreter 기능 테스트
- ✅ 네트워크 액세스 기능을 갖춘 커스텀 Code Interpreter 생성
- ✅ 동적 Python 실행을 위해 Code Interpreter를 Strands Agents와 통합
- ✅ 데이터 분석, 웹 요청, 파일 작업을 포함한 복잡한 작업 실행

## AgentCore Code Interpreter의 주요 이점

- **동적 코드 실행**: AI 에이전트 워크플로우 내에서 Python 코드를 온디맨드로 실행
- **보안 샌드박스 환경**: 구성 가능한 네트워크 액세스를 통한 격리된 실행
- **세션 상태 관리**: 여러 실행에 걸쳐 변수와 컨텍스트 유지
- **원활한 에이전트 통합**: Strands Agents 내에서 네이티브 도구로 작동
- **유연한 구성**: 특정 사용 사례와 요구사항에 맞게 인터프리터 커스터마이징